## Initial Setup

In [ ]:
!nvidia-smi

Fri Mar 26 17:51:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Imports

In [ ]:
import os
import numpy as np
from PIL import Image
import pdb
import time

import torch
import torchvision   
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

## Hyperparams

In [ ]:
N_EPOCHS = 100
in_features = 3 # RGB channels

learningRate = 0.1
weightDecay = 5e-5

BATCH_SIZE = 256
feat_dim = 4096

## Residual Block

In [ ]:
class Block(nn.Module):
    def __init__(self, num_layers, in_channels, out_channels, identity_downsample=None, stride=1):
        super(Block, self).__init__()
        self.num_layers = num_layers
        self.expansion = 1
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * self.expansion, kernel_size=1, stride=1, padding=0)
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample

    def forward(self, x):
        identity = x
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x

In [ ]:
class ResNet(nn.Module):
    def __init__(self, num_layers, block, image_channels, num_classes,feat_dim = 1024):
        super(ResNet, self).__init__()
        self.num_classes = num_classes
        self.expansion = 1
        layers = [3, 4, 6, 3]
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # ResNetLayers
        self.layer1 = self.make_layers(num_layers, block, layers[0], intermediate_channels=64, stride=1)
        self.layer2 = self.make_layers(num_layers, block, layers[1], intermediate_channels=128, stride=2)
        self.layer3 = self.make_layers(num_layers, block, layers[2], intermediate_channels=256, stride=2)
        self.layer4 = self.make_layers(num_layers, block, layers[3], intermediate_channels=512, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        # self.fc = nn.Linear(512 * self.expansion, num_classes)

        self.linear = nn.Linear(512 * self.expansion, feat_dim)
        self.relu = nn.ReLU(inplace=True)
        self.linear_output = nn.Linear(512 * self.expansion,num_classes)

    def forward(self, x, return_embedding=False):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)

        embedding = x
        embedding_out = self.relu(self.linear(embedding))
        output = self.linear_output(embedding)
        if return_embedding:
            return embedding_out,output
        else:
            return output  

    def make_layers(self, num_layers, block, num_residual_blocks, intermediate_channels, stride):
        layers = []

        identity_downsample = nn.Sequential(nn.Conv2d(self.in_channels, intermediate_channels*self.expansion, kernel_size=1, stride=stride),
                                            nn.BatchNorm2d(intermediate_channels*self.expansion))
        layers.append(block(num_layers, self.in_channels, intermediate_channels, identity_downsample, stride))
        self.in_channels = intermediate_channels * self.expansion # 256
        for i in range(num_residual_blocks - 1):
            layers.append(block(num_layers, self.in_channels, intermediate_channels)) # 256 -> 64, 64*4 (256) again
        return nn.Sequential(*layers)


## Load in Data (with augmentation)

In [ ]:
transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224,224)),
    torchvision.transforms.ColorJitter(hue=.05, saturation=.05),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(20),
    torchvision.transforms.ToTensor()
])
val_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224,224)),
    torchvision.transforms.ColorJitter(hue=.05, saturation=.05),
    torchvision.transforms.ToTensor(),
])

train_dataset = torchvision.datasets.ImageFolder(root='train_data/', 
                                                 transform=transforms)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, 
                                               shuffle=True, num_workers=4)
# deal with class_to_idx problem
idx_to_class = {v: k for k, v in train_dataset.class_to_idx.items()}


val_dataset = torchvision.datasets.ImageFolder(root='val_data/', 
                                               transform=val_transforms)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, 
                                             shuffle=False, num_workers=4)


### Extra setup

In [ ]:
num_classes = len(train_dataset.classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = ResNet(34, Block, in_features, num_classes)
model = model.to(device)
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate,  momentum=0.9, weight_decay=weightDecay, nesterov=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', verbose=True)

In [ ]:
# Training function
def train_epoch(model, train_loader, criterion, optimizer, epoch_num):
    print(f"Epoch {epoch_num}...")
    model.train()

    running_loss = 0.0
    total_predictions = 0.0
    correct_predictions = 0.0
    
    start_time = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):   
        optimizer.zero_grad() 
        data = data.to(device)
        target = target.to(device)

        outputs = model(data)
        loss = criterion(outputs, target.long())
        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_predictions += target.size(0)
        correct_predictions += (predicted == target).sum().item()

        loss.backward() # Calculate gradients
        optimizer.step() # Apply gradient descent step
    
    end_time = time.time()
    
    running_loss /= len(train_loader)
    acc = (correct_predictions/total_predictions)*100.0
    print('Training Loss: ', running_loss, 'Time: ',end_time - start_time, 's')
    print('Training Accuracy: ', acc, '%')
    print('Train time (min):', (end_time - start_time)/60)
    print()
    return running_loss, acc

In [ ]:
# Validation/ Evaluation Function
def val_model(model, val_loader, criterion, scheduler):
    print('Validating...')
    with torch.no_grad():
        model.eval()

        running_loss = 0.0
        total_predictions = 0.0
        correct_predictions = 0.0

        start_time = time.time()
        for batch_idx, (data, target) in enumerate(val_loader):   
            data = data.to(device)
            target = target.to(device)

            outputs = model(data)

            _, predicted = torch.max(outputs.data, 1)
            total_predictions += target.size(0)
            correct_predictions += (predicted == target).sum().item()

            loss = criterion(outputs, target).detach()
            running_loss += loss.item()

        scheduler.step(running_loss)

        end_time = time.time()
        running_loss /= len(val_loader)
        acc = (correct_predictions/total_predictions)*100.0
        print('Validation Loss: ', running_loss)
        print('Validation Accuracy: ', acc, '%')
        print('Validation time (min):', (end_time - start_time)/60)
        print()
        return running_loss, acc


In [ ]:
# Run the model
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []
trialNum = 2
best_epoch = 0
winner = ""

current_best = 0

for epoch in range(N_EPOCHS): 
    train_loss, train_acc = train_epoch(model, train_dataloader, criterion, optimizer, epoch)
    val_loss, val_acc = val_model(model, val_dataloader, criterion, scheduler=scheduler)
    
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    if val_acc > current_best:
      # Get old model to be deleted
      old_model = f"Trial{trialNum}-epoch{best_epoch}.pth"

      # Save new best model
      current_best = val_acc
      best_epoch = epoch
      winner = f"Trial{trialNum}-epoch{epoch}"
      filename = save_location_raw+winner+".pth"
      torch.save(model.state_dict(), filename)

      # Delete old model if exists
      if os.path.exists(old_model):
        os.remove(old_model)

    print('='*20)

In [ ]:
winning_model = model
winning_model.load_state_dict(torch.load(save_location_raw+winner+".pth"))
winning_model.eval()

## Dataset & Data Loader Definition for Testing

In [ ]:
# Test Dataset definition
class TestDataset(Dataset):
    # load the dataset
    def __init__(self, path, transform):
        self.path = path
        self.X = sorted([x.split('.')[0] for x in os.listdir(path)], key=int)
        self.transform = transform  
        
    # number of rows in the dataset
    def __len__(self):
        return len(self.X)
 
    # get a row at an index
    def __getitem__(self, index):
        x = self.transform(Image.open(self.path+self.X[index]+".jpg"))
        return x

In [ ]:
test_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224,224)),
    torchvision.transforms.ToTensor(),
    # torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

test_dataset = TestDataset("test_data/", test_transforms)
test_dataloader_args = dict(shuffle=False, batch_size=BATCH_SIZE, num_workers=4, pin_memory=True) if torch.cuda.is_available()\
                else dict(shuffle=False, batch_size=1)
test_dataloader = torch.utils.data.DataLoader(test_dataset, **test_dataloader_args)

In [ ]:
def test_on_final_model(model, test_loader, criterion):
  print('Final Testing...')
  with torch.no_grad():
    model.eval()

    p = []
    i = 0
    for batch_idx, (data) in enumerate(test_loader):
      data = data.to(device)
      outputs = model(data)

      _, predicted = torch.max(outputs.data, 1)
      p.append(predicted.cpu().numpy())# get predictions back to cpu. no more gpu

    return p # This will be written to csv

In [ ]:
p = test_on_final_model(winning_model, test_dataloader, criterion)
print('Done')

Final Testing...
Done


## Write submission file

In [ ]:
with open(save_location_raw+winner+".csv", 'w') as f:
  f.write(f"id,label\n")
  for indx, pred in enumerate(np.concatenate(p)):
    f.write(f"{indx}.jpg,{idx_to_class[pred]}\n")

In [ ]:
print(winner)